In [7]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas as pd
#Can't use keras on 129.114.32.33:8000
#from keras import layers, models, optimizers
import datetime, os

#to-do list
#1. Record the amount of time a classifier takes: start(timestamp) - end(timestamp), put it in .pkl.gz

#Observations:
# 1. RF gives highest accuracy, but takes a lot of time to train: 25 minutes and 15 minutes
# 2. Neural Network is weakest
# 3. NB gives satisfactory results within a minute.

trainDF = pd.concat([pd.read_pickle('../../data/2015/MasterData_2015.pkl.gz'),
                    pd.read_pickle('../../data/2014/MasterData_2014.pkl.gz'),
                    pd.read_pickle('../../data/2013/MasterData_2013.pkl.gz'),
                    pd.read_pickle('../../data/2012/MasterData_2012.pkl.gz')])
                                   
trainDF = trainDF[trainDF.TEXT.notna()]
trainDF['text'] = trainDF['TEXT'].astype(str)
trainDF['label'] = trainDF['NTEE'].astype(str)

# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])

# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return [metrics.accuracy_score(predictions, valid_y), 
            metrics.precision_score(predictions, valid_y, average='weighted'), 
            metrics.recall_score(predictions, valid_y, average='weighted')]

results = pd.DataFrame(columns=['classifier', 'accuracy', 'precision', 'recall', 'time'])

# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
results.loc[len(results)] = ["NB, Count Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
results.loc[len(results)] = ["NB, WordLevel TF-IDF", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
results.loc[len(results)] = ["NB, N-Gram Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
results.loc[len(results)] = ["NB, CharLevel Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NB, CharLevel Vectors: ", accuracy)

# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
results.loc[len(results)] = ["RF, Count Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
results.loc[len(results)] = ["RF, WordLevel TF-IDF", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("RF, WordLevel TF-IDF: ", accuracy)

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)


def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 


classifier = create_model_architecture(xtrain_count.shape[1])
accuracy = train_model(classifier, xtrain_count, train_y, xvalid_count, is_neural_net=True)
results.loc[len(results)] = ["NN, Count Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NN, Count Vectors",  accuracy)

classifier = create_model_architecture(xtrain_tfidf_ngram_chars.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars, is_neural_net=True)
results.loc[len(results)] = ["NN, CharLevel Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NN, CharLevel Vectors",  accuracy)

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])
accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
results.loc[len(results)] = ["NN, Ngram Level TF IDF Vectors", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NN, Ngram Level TF IDF Vectors",  accuracy)

classifier = create_model_architecture(xtrain_tfidf.shape[1])
accuracy = train_model(classifier, xtrain_tfidf, train_y, xvalid_tfidf, is_neural_net=True)
results.loc[len(results)] = ["NN, World Level TF-IDF", accuracy[0], accuracy[1], accuracy[2], datetime.datetime.now()]
print("NN, World Level TF-IDF",  accuracy)


if(os.path.exists('../../data/results/classifier_results.pkl.gz')):
    results = pd.concat([pd.read_pickle('../../data/results/classifier_results.pkl.gz'), results]).drop_duplicates()

results.to_pickle('../../data/results/classifier_results.pkl.gz')

/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


NB, Count Vectors:  [0.6451042830229573, 0.6659661298547271, 0.6451042830229573]
NB, WordLevel TF-IDF:  [0.6196295493572965, 0.6661719314247055, 0.6196295493572965]
NB, N-Gram Vectors:  [0.5338579439044586, 0.6007282836695058, 0.5338579439044586]
NB, CharLevel Vectors:  [0.5761824577782965, 0.622573367036641, 0.5761824577782965]
RF, Count Vectors:  [0.8889426096041524, 0.892335117353861, 0.8889426096041524]
RF, WordLevel TF-IDF:  [0.8888648071889431, 0.8923074915133422, 0.8888648071889431]


In [10]:
temp = pd.concat([pd.read_pickle('../../data/results/classifier_results.pkl.gz'),
                pd.read_pickle('../../data/results/classifier_results1.pkl.gz')]).drop_duplicates()

temp.to_pickle('../../data/results/classifier_results.pkl.gz')
   
print(pd.read_pickle('../../data/results/classifier_results.pkl.gz'))

                       classifier  accuracy  precision    recall  \
0               NB, Count Vectors  0.645104   0.665966  0.645104   
1            NB, WordLevel TF-IDF  0.619630   0.666172  0.619630   
2              NB, N-Gram Vectors  0.533858   0.600728  0.533858   
3           NB, CharLevel Vectors  0.576182   0.622573  0.576182   
4               RF, Count Vectors  0.888943   0.892335  0.888943   
5            RF, WordLevel TF-IDF  0.888865   0.892307  0.888865   
0               NB, Count Vectors  0.645882   0.666515  0.645882   
1            NB, WordLevel TF-IDF  0.619741   0.666131  0.619741   
2              NB, N-Gram Vectors  0.535358   0.600996  0.535358   
3           NB, CharLevel Vectors  0.579117   0.624497  0.579117   
4  NN, Ngram Level TF IDF Vectors  0.062820   1.000000  0.062820   
5          NN, World Level TF-IDF  0.062820   1.000000  0.062820   
6               NN, Count Vectors  0.062820   1.000000  0.062820   
7           NN, CharLevel Vectors  0.062820   1.